<a href="https://colab.research.google.com/github/JefNtungila/DS-Unit-2-Classification-2/blob/master/Jef_Ntungila_class_imbalance_roc_auc_assignment_lecture_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders #instanlling encoders

     |████████████████████████████████| 92kB 3.9MB/s 


In [0]:
#importing data 

history_location = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Classification-2/master/data/lending-club-subset.csv'
current_location = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Classification-2/master/data/primaryMarketNotes_browseNotes_1-RETAIL.csv'

In [0]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

# Stratified sample, 10% of expired Lending Club loans, grades A-D
# Source: https://www.lendingclub.com/info/download-data.action
history = pd.read_csv(history_location)
history['issue_d'] = pd.to_datetime(history['issue_d'], infer_datetime_format=True)

# Current loans available for manual investing, June 17, 2019
# Source: https://www.lendingclub.com/browse/browse.action
current = pd.read_csv(current_location)

In [4]:
# Calculate percent of each loan repaid
history['percent_paid'] = history['total_pymnt'] / history['funded_amnt']

# See percent paid for charged off vs fully paid loans
history.groupby('loan_status')['percent_paid'].describe()

,count,mean,std,min,25%,50%,75%,max
loan_status,,,,,,,,
Charged Off,22477.0,0.546541,0.278077,0.0,0.327761,0.514006,0.742090,1.490546
Fully Paid,105857.0,1.146904,0.089759,1.0,1.083836,1.132098,1.195798,1.623628


In [0]:
#we explore some baselines using a monte carlo approach

In [0]:
# Transform earliest_cr_line to an integer:
# How many days the earliest credit line was open, before the loan was issued.
# For current loans available for manual investing, assume the loan will be issued today.

history['earliest_cr_line'] = pd.to_datetime(history['earliest_cr_line'], infer_datetime_format=True)
history['earliest_cr_line'] = history['issue_d'] - history['earliest_cr_line']
history['earliest_cr_line'] = history['earliest_cr_line'].dt.days

current['earliest_cr_line'] = pd.to_datetime(current['earliest_cr_line'], infer_datetime_format=True)
current['earliest_cr_line'] = pd.Timestamp.today() - current['earliest_cr_line']
current['earliest_cr_line'] = current['earliest_cr_line'].dt.days

In [0]:
# Use Python sets to compare the historical columns & current columns

common_columns = set(history.columns) & set(current.columns)
just_history = set(history.columns) - set(current.columns)
just_current = set(current.columns) - set(history.columns)

In [0]:
# Train on the historical data.
# For features, use only the common columns shared by the historical & current data.
# For the target, use `loan_status` ('Fully Paid' or 'Charged Off')

features = list(common_columns)
target = 'loan_status'
X = history[features]
y = history[target]

In [9]:
# Do train/validate/test 3-way split

from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=20000, stratify=y, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=20000, 
    stratify=y_trainval, random_state=42)

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (88334, 106)
y_train shape (88334,)
X_val shape (20000, 106)
y_val shape (20000,)
X_test shape (20000, 106)
y_test shape (20000,)


In [10]:
%%time

def wrangle(X):
    X = X.copy()

    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
        
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)

    # Transform features with many nulls to binary flags
    many_nulls = ['member_id', 'sec_app_mths_since_last_major_derog', 'sec_app_revol_util', 
                  'sec_app_open_act_il', 'sec_app_chargeoff_within_12_mths', 'sec_app_mort_acc', 
                  'sec_app_fico_range_high', 'revol_bal_joint', 'sec_app_collections_12_mths_ex_med', 
                  'sec_app_inq_last_6mths', 'sec_app_num_rev_accts', 'sec_app_open_acc', 
                  'sec_app_earliest_cr_line', 'sec_app_fico_range_low', 'annual_inc_joint', 
                  'dti_joint', 'desc', 'mths_since_last_record', 'mths_since_recent_bc_dlq', 
                  'mths_since_last_major_derog', 'mths_since_recent_revol_delinq', 'il_util', 
                  'mths_since_rcnt_il', 'all_util', 'open_il_12m', 'total_bal_il', 'open_il_24m', 
                  'inq_last_12m', 'open_rv_12m', 'max_bal_bc', 'open_act_il', 'open_rv_24m', 'inq_fi', 
                  'open_acc_6m', 'total_cu_tl', 'mths_since_last_delinq', 'mths_since_recent_inq', 
                  'num_tl_120dpd_2m', 'mo_sin_old_il_acct', 'emp_title', 'emp_length', 'pct_tl_nvr_dlq', 
                  'avg_cur_bal', 'mo_sin_rcnt_rev_tl_op', 'num_actv_bc_tl', 'num_il_tl', 
                  'mo_sin_rcnt_tl', 'total_rev_hi_lim', 'tot_cur_bal', 'num_bc_tl', 'num_tl_30dpd', 
                  'num_op_rev_tl', 'mo_sin_old_rev_tl_op', 'total_il_high_credit_limit', 
                  'num_rev_tl_bal_gt_0', 'num_accts_ever_120_pd', 'num_tl_90g_dpd_24m', 
                  'num_tl_op_past_12m', 'num_rev_accts', 'tot_hi_cred_lim', 'tot_coll_amt', 
                  'num_actv_rev_tl', 'bc_util', 'percent_bc_gt_75', 'bc_open_to_buy', 
                  'mths_since_recent_bc', 'num_bc_sats', 'num_sats', 'total_bc_limit', 
                  'acc_open_past_24mths', 'total_bal_ex_mort', 'mort_acc', 'title']
    
    for col in many_nulls:
        X[col] = X[col].isnull()
    
    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mean())
            
    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns='member_id')  # member_id is always null
    X = X.drop(columns='title')  # Duplicative of purpose
    X = X.drop(columns='grade')  # Duplicative of sub_grade
    X = X.drop(columns='zip_code') # High cardinality

    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(X_train)
X_val   = wrangle(X_val)
X_test  = wrangle(X_test)

CPU times: user 3 s, sys: 149 ms, total: 3.15 s
Wall time: 3.15 s


In [12]:
%%time
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
)

pipeline.fit(X_train, y_train)

CPU times: user 44.6 s, sys: 183 ms, total: 44.8 s
Wall time: 23.6 s


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score 

y_pred = pipeline.predict(X_val)
accuracy_score(y_val, y_pred)

0.8242

In [15]:
from sklearn.metrics import roc_auc_score

y_pred_proba = pipeline.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred_proba)

0.6837494792044388